 - mistral-instruct inference tutorial https://huggingface.co/blog/Andyrasika/mistral-7b-empowering-conversation
 - https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2

In [1]:
!pip install -q -U git+https://github.com/huggingface/transformers bitsandbytes peft datasets accelerate trl --use-deprecated=legacy-resolver

ERROR: pip's legacy dependency resolver does not consider dependency conflicts when selecting packages. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you'll have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you'll have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you'll have dask 2023.12.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you'll have distributed 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you'll have dask 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires distributed==2023.7.1, but you'll have distributed 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires pa

In [2]:
from transformers import AutoModelForCausalLM
import torch
from transformers import BitsAndBytesConfig

from transformers import AutoTokenizer
import numpy as np
import pickle
from accelerate.utils import release_memory
from accelerate import load_checkpoint_and_dispatch
import gc
import time

In [3]:
device = "cuda"

In [4]:
def load_checkpoint_model(path):
    bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
    )
    
    model = AutoModelForCausalLM.from_pretrained(
            path,
            load_in_4bit=True,
            #quantization_config=bnb_config,
            torch_dtype=torch.bfloat16,
            device_map="auto", # "cuda" to large, "cpu" somehow does not put LinearFP4 Layer on device
        )

    return model

base_model = "/kaggle/input/mistral-finetuning-swarm-v1/results/checkpoint-400"
model = load_checkpoint_model(base_model)
tokenizer = AutoTokenizer.from_pretrained(base_model)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
# del model
# model = None
# gc.collect()
# torch.cuda.empty_cache()
# tokenizer, model = release_memory(tokenizer, model)

In [6]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=1024, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=Fa

In [7]:
def generate_text(model, instruction: str, max_new_tokens=1000, device="cuda", tokenizer=tokenizer) -> str:
  messages = [
      {"role": "user", "content": instruction},
  ]

  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt") # wraps text with special tokens depending on role (assitant or user)
  model_inputs = encodeds.to(device)
  generated_ids = model.generate(model_inputs, max_new_tokens=max_new_tokens, do_sample=False)
  decoded = tokenizer.decode(generated_ids[0][len(model_inputs[0]):], skip_special_tokens=True) # [0]: first dimension has size 1
  return decoded

In [8]:
text = "The environment is constructed as a rectangular space with a length of 4.36 meters, width of 3.82 meters, and height of 2.05 meters.There are 3 lights distributed evenly in the arena. Placed within a 1.77-meter radius around the center are 17 robots. The swarm's goal is to cover an area of 2.90 by 4.32 meters while ensuring connectivity."
req = "\nGenerate the xml configuration for this mission."

start_time = time.time()
response = generate_text(model, text+req, max_new_tokens=5000,device=device)
end_time = time.time()
end_time - start_time

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


87.1864333152771

In [9]:
print(text+req)

The environment is constructed as a rectangular space with a length of 4.36 meters, width of 3.82 meters, and height of 2.05 meters.There are 3 lights distributed evenly in the arena. Placed within a 1.77-meter radius around the center are 17 robots. The swarm's goal is to cover an area of 2.90 by 4.32 meters while ensuring connectivity.
Generate the xml configuration for this mission.


In [10]:
print(response)

<?xml version="1.00" ?>
<config>
  <swarm-elems>
    <entity quantity="17" max_trials="100">
      <e-puck id="epuck">
        <controller config="automode_bt"/>
      </e-puck>
    </entity>
  </swarm-elems>
  <env-elems>
    <light id="light_0" position="0.86,-1.12,0.00" orientation="360,0,0" color="yellow" intensity="2.92" medium="leds"/>
    <light id="light_1" position="0.91,-0.84,0.00" orientation="360,0,0" color="yellow" intensity="4.44" medium="leds"/>
    <light id="light_2" position="0.84,-0.84,0.00" orientation="360,0,0" color="yellow" intensity="3.42" medium="leds"/>
    <box id="wall_0" size="0.01,3.82,0.10" movable="false">
      <body position="0,-2.18,0.00" orientation="90.00,0.00,0.00"/>
    </box>
    <box id="wall_1" size="0.01,4.36,0.10" movable="false">
      <body position="1.91,0,0.00" orientation="0.00,0.00,0.00"/>
    </box>
    <box id="wall_2" size="0.01,3.82,0.10" movable="false">
      <body position="0,2.18,0" orientation="90.00,0.00,0.00"/>
    </box>
   

## Generate Evaluation Data

In [11]:
!python -m pip install git+https://github.com/julianjandeleit/swarm_descriptions.git@df7cb8adb36573bfef7c2bb13ab771c87fb9d4f5

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/julianjandeleit/swarm_descriptions.git (to revision df7cb8adb36573bfef7c2bb13ab771c87fb9d4f5) to /tmp/pip-req-build-hw1lgrve
  Running command git clone --filter=blob:none --quiet https://github.com/julianjandeleit/swarm_descriptions.git /tmp/pip-req-build-hw1lgrve
  Running command git rev-parse -q --verify 'sha^df7cb8adb36573bfef7c2bb13ab771c87fb9d4f5'
  Running command git fetch -q https://github.com/julianjandeleit/swarm_descriptions.git df7cb8adb36573bfef7c2bb13ab771c87fb9d4f5
  Running command git checkout -q df7cb8adb36573bfef7c2bb13ab771c87fb9d4f5
  Resolved https://github.com/julianjandeleit/swarm_descriptions.git to commit df7cb8adb36573bfef7c2bb13ab771c87fb9d4f5
  Installing build dependencies ... - \ | / - \ | / done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for swarm_descriptions: filename=swarm_descriptions-0.1.0-py3-none-any.whl size=28692 sha256=a9bdb500ca2db

In [12]:
from swarm_descriptions import utils
from swarm_descriptions.configfiles import Configurator, ET, config_to_string
from swarm_descriptions.mission_elements import MissionParams, get_generators
import random
import pandas as pd
import time

In [13]:
template = """<?xml version="1.0" ?>
<argos-configuration>
    <!-- ************* -->
    <!-- * Framework * -->
    <!-- ************* -->
    <framework>
        <experiment length="120" ticks_per_second="10" random_seed="0"/>
    </framework>

    <!-- ****************** -->
    <!-- * Loop functions * -->
    <!-- ****************** -->
    <loop_functions library="/opt/argos/AutoMoDe-loopfunctions/build/loop-functions/custom/libcustom_loopfunc.so" label="template">
        <loop-placeholder></loop-placeholder>
    </loop_functions>

    <!-- *************** -->
    <!-- * Controllers * -->
    <!-- *************** -->
    <controllers>
    <automode_controller_bt id="automode_bt" library="/opt/argos/AutoMoDe/build/src/libautomode_bt.so">
            <actuators>
                <epuck_wheels implementation="default" noise_std_dev="0.05"/>
                <epuck_rgb_leds implementation="default" medium="leds"/>
                <epuck_range_and_bearing implementation="medium" medium="rab" data_size="4" range="0.7"/>
            </actuators>
            <sensors>
                <epuck_proximity implementation="default" show_rays="false" noise_level="0.05" calibrated="true"/>
                <epuck_range_and_bearing implementation="medium" medium="rab" data_size="4" nois_std_deviation="1.5" loss_probability="0.85" calibrated="true"/>
                <epuck_light implementation="default" show_rays="false" noise_level="0.05" calibrated="true"/>
                <epuck_ground implementation="rot_z_only" noise_level="0.05" calibrated="true"/>
                <epuck_omnidirectional_camera implementation="rot_z_only" medium="leds" show_rays="false"/>
            </sensors>
      <params bt-config="--nroot 3 --nchildroot 1 --n0 0 --nchild0 2 --n00 6 --c00 5 --p00 0.26 --n01 5 --a01 1 --p01 0"/>
    </automode_controller_bt>
    </controllers>

    <!-- ********* -->
    <!-- * Arena * -->
    <!-- ********* -->
    <arena size="10, 10, 1" center="0,0,0">

        <!-- Change the floor here -->
        <floor id="floor" source="loop_functions" pixels_per_meter="300"/>
        <!-- Change the light here -->
        <!-- Please note that this light has an intensity equal to 0 -->
        <!-- and therefore does not affect the experiment. -->
        <!-- ARGoS requires at least one light when using the light sensor -->
        <light id="light" position="0,-1.35,0.45" orientation="0,0,0" color="yellow" intensity="5.0" medium="leds"/>

        <environment-placeholder></environment-placeholder>
        <!-- ********** -->
        <!-- * E-Puck * -->
        <!-- ********** -->

        <!-- The epucks are distributed by the AutoMoDeLoopFunctions. -->
        <distribute>
                <robots-placeholder></robots-placeholder>
        </distribute>
    </arena>

    <!-- ******************* -->
    <!-- * Physics engines * -->
    <!-- ******************* -->
    <physics_engines>
        <dynamics2d id="dyn2d" />
    </physics_engines>

    <!-- ********* -->
    <!-- * Media * -->
    <!-- ********* -->
    <media>
        <led id="leds" grid_size="1,1,1"/>
        <range_and_bearing id="ircom"/>
        <range_and_bearing id="rab"/>
    </media>

    <!-- ***************** -->
    <!-- * Visualization * -->
    <!-- ***************** -->
  <visualization>
        <qt-opengl>
            <camera>
                <placement idx="0"
                           position="0,0,4.5"
                           look_at="0,0,0"
                           lens_focal_length="30" />
            </camera>
        </qt-opengl>
  </visualization>

</argos-configuration>
"""

In [14]:
template = ET.fromstring(template)

In [15]:
config_params = response
config_params = ET.fromstring(config_params)
argos_config = Configurator().convert_config_params(config_params, template)
argos_config = config_to_string(argos_config)

In [16]:
print(argos_config)

<?xml version="1.00" ?>
<argos-configuration>
  <framework>
    <experiment length="120" ticks_per_second="10" random_seed="0"/>
  </framework>
  <loop_functions library="/opt/argos/AutoMoDe-loopfunctions/build/loop-functions/custom/libcustom_loopfunc.so" label="template">
    <spawnCircle position="0,0,0" radius="1.77"/>
    <objective type="distribution">
      <objective-params area="2.90,4.32" connection_range="0.89"/>
    </objective>
  </loop_functions>
  <controllers>
    <automode_controller_bt id="automode_bt" library="/opt/argos/AutoMoDe/build/src/libautomode_bt.so">
      <actuators>
        <epuck_wheels implementation="default" noise_std_dev="0.05"/>
        <epuck_rgb_leds implementation="default" medium="leds"/>
        <epuck_range_and_bearing implementation="medium" medium="rab" data_size="4" range="0.70"/>
      </actuators>
      <sensors>
        <epuck_proximity implementation="default" show_rays="false" noise_level="0.05" calibrated="true"/>
        <epuck_range_a

In [17]:
def generate_for_checkpoint(checkpoint="checkpoint-400", n_rows = 10):
    print(f"generating for checkpoint {checkpoint}")
    base_model = f"/kaggle/input/mistral-finetuning-swarm-v1/results/{checkpoint}"
    try:
        # Remove old model to make space for new one in ram
        global model
        del model
        model = None
        gc.collect()
        time.sleep(1)
        torch.cuda.empty_cache()
        model = release_memory(model)
    except Exception as e:
        print(f"ERROR: {e}")
    model = load_checkpoint_model(base_model)
    tokenizer = AutoTokenizer.from_pretrained(base_model)

    rows = []
    generators = get_generators()
    start_time = time.time()
    for n in range(n_rows):
        mission = MissionParams.sample(*generators)
        conf = config_to_string(mission.configure())
        conf = utils.truncate_floats(conf)
        desc = random.sample(mission.describe(),1)[0]  
        desc = utils.truncate_floats(desc)
        rows.append({"description": desc, "configuration": conf, "parameters": mission})

    end_time = time.time()
    dataset = pd.DataFrame(rows)
    dataset.head()

    req = "\nGenerate the xml configuration for this mission."
    responses = []
    start_time = time.time()
    for i, row in dataset.iterrows():
        response = row.description+req
        response = generate_text(model, response, max_new_tokens=2500,device="cpu",tokenizer=tokenizer)
        responses.append(response)
        print(f"generated response for description {i}")
    end_time = time.time()
    print(f"took {end_time - start_time}s")
    dataset["response"] = responses
    
    return dataset

In [18]:
# this takes too long so should be run only if checkpoint_numbers is empty
# dataset_detailed = generate_for_checkpoint(f"checkpoint-{400}", n_rows=200)
# dataset_detailed.to_pickle("eval_dataset_detailed.pkl")

In [19]:
checkpoint_numbers = [a+b for a in [25, 50, 75, 100] for b in [0, 100, 200, 300]]
checkpoint_numbers.sort()
checkpoint_numbers

[25, 50, 75, 100, 125, 150, 175, 200, 225, 250, 275, 300, 325, 350, 375, 400]

In [20]:
datasets = dict()
for num in checkpoint_numbers:
    del model
    model = None
    gc.collect()
    torch.cuda.empty_cache()
    tokenizer, model = release_memory(tokenizer, model)
    datasets[f"checkpoint-{num}"] = generate_for_checkpoint(f"checkpoint-{num}", n_rows=14)
    
with open('datasets.pickle', 'wb') as handle:
    pickle.dump(datasets, handle, protocol=pickle.HIGHEST_PROTOCOL)


generating for checkpoint checkpoint-25


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1665: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


generated response for description 0


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

generated response for description 10


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


generated response for description 11


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


generated response for description 12
generated response for description 13
took 1576.7528460025787s


In [21]:
with open('datasets.pickle', 'rb') as handle:
    test_load = pickle.load(handle)

In [22]:
test_load

{'checkpoint-25':                                           description  \
 0   The circular arena, constructed with 19 walls,...   
 1   The environment is a circle made out of 6 wall...   
 2   In this setting, a rectangle is formed with di...   
 3   The environment features a rectangle with dime...   
 4   The circular arena, having a radius of 2.93 me...   
 5   The circular arena, constructed with 12 walls,...   
 6   The circular arena, having a radius of 1.73 me...   
 7   The circular arena, constructed with 17 walls,...   
 8   The rectangular area has dimensions 1.74 m x 3...   
 9   A circle with 15 walls forms the structure of ...   
 10  The environment is constructed as a rectangula...   
 11  The environment is a circular arena with radiu...   
 12  With a length of 4.01 meters, width of 6.62 me...   
 13  The environment is constructed as a rectangula...   
 
                                         configuration  \
 0   <?xml version="1.00" ?>\n<config>\n  <swarm-el..